In [ ]:
import pandas as pd
import pymysql

df = pd.read_csv("alquiler.csv")

# Crear tipo_vivienda a partir de las columnas booleanas de Pablo
df["tipo_vivienda"] = df.apply(lambda row: (
    "ático" if row["atico"] else
    "apartamento" if row["apartamento"] else
    "estudio" if row["estudio"] else
    "dúplex" if row["duplex"] else
    "chalet" if row["chalet"] else
    "finca" if row["finca"] else
    "loft" if row["loft"] else
    "piso"
), axis=1)

 

conexion = pymysql.connect(
    host="localhost",
    user="root",
    password="Destroyerl0",
    database="propiedades_de_alquiler",
    charset="utf8mb4"
)

cursor = conexion.cursor()

# Insertar datos (sin descripcion_ampliada y con superficie_util como None)
for _, fila in df.iterrows():
    sql = """
    INSERT INTO anuncios (
        titulo, precio, ubicacion, habitaciones, banos, metros,
        tipo_vivienda, superficie_construida, superficie_util,
        link
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    datos = (
        fila["titulo"],
        float(fila["precio"]) if not pd.isna(fila["precio"]) else None,
        fila["ubicacion"],
        int(fila["habitaciones"]) if not pd.isna(fila["habitaciones"]) else None,
        int(fila["baños"]) if not pd.isna(fila["baños"]) else None,
        int(fila["metros"]) if not pd.isna(fila["metros"]) else None,
        fila["tipo_vivienda"],
        int(fila["superficie_construida"]) if not pd.isna(fila["superficie_construida"]) else None,
        None,  # superficie_util no está en el archivo
        fila["link"]
    )
    cursor.execute(sql, datos)


conexion.commit()
cursor.close()
conexion.close()